In [ ]:
print("ML Expt 7")
print("Dhwaj Jain S012")

ML Expt 7
Dhwaj Jain S012


In [ ]:
pip install pgmpy


In [ ]:
from pgmpy.models import DiscreteBayesianNetwork as DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
#structure of the Bayesian Network

model_structure = DiscreteBayesianNetwork([
    ('Burglary', 'Alarm'),
    ('Earthquake', 'Alarm'),
    ('Alarm', 'JohnCalls'),
    ('Alarm', 'MerryCalls')
])

In [ ]:
#CPDs (Conditional Probabilty Distributions)
cpd_burglary = TabularCPD(
    variable="Burglary",
    variable_card=2,
    values=[[.999], [.001]]
)

print(cpd_burglary)

+-------------+-------+
| Burglary(0) | 0.999 |
+-------------+-------+
| Burglary(1) | 0.001 |
+-------------+-------+


In [ ]:
cpd_earthquake=TabularCPD(
    variable="Earthquake",
    variable_card=2,
    values=[[.998], [.002]]
)

In [ ]:
print(cpd_earthquake)

+---------------+-------+
| Earthquake(0) | 0.998 |
+---------------+-------+
| Earthquake(1) | 0.002 |
+---------------+-------+


In [ ]:
cpd_JohnCall= TabularCPD(
    variable="JohnCalls",
    variable_card=2,
    values=[[.95, .1], # P(JohnCalls=True | Alarm=True), P(JohnCalls=True | Alarm=False)
            [.05, .9]], # P(JohnCalls=False | Alarm=True), P(JohnCalls=False | Alarm=False)
    evidence=["Alarm"],
    evidence_card=[2]
)

In [ ]:
print(cpd_JohnCall)

+--------------+----------+----------+
| Alarm        | Alarm(0) | Alarm(1) |
+--------------+----------+----------+
| JohnCalls(0) | 0.95     | 0.1      |
+--------------+----------+----------+
| JohnCalls(1) | 0.05     | 0.9      |
+--------------+----------+----------+


In [ ]:
cpd_MerryCalls = TabularCPD(variable = 'MerryCalls', variable_card = 2, values = [ [.99, 0.3], [.01, 0.7]], evidence = ['Alarm'], evidence_card = [2])

In [ ]:
print(cpd_MerryCalls)

+---------------+----------+----------+
| Alarm         | Alarm(0) | Alarm(1) |
+---------------+----------+----------+
| MerryCalls(0) | 0.99     | 0.3      |
+---------------+----------+----------+
| MerryCalls(1) | 0.01     | 0.7      |
+---------------+----------+----------+


In [ ]:
cpd_alarm = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[.999, .71, .06, .05], # P(Alarm=True | Burglary=True, Earthquake=True), P(Alarm=True | Burglary=True, Earthquake=False), P(Alarm=True | Burglary=False, Earthquake=True), P(Alarm=True | Burglary=False, Earthquake=False)
            [.001, .29, .94, .95]],# P(Alarm=False | Burglary=True, Earthquake=True), P(Alarm=False | Burglary=True, Earthquake=False), P(Alarm=False | Burglary=False, Earthquake=False)
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2]
)

In [ ]:
print(cpd_alarm)

+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.999         | 0.71          | 0.06          | 0.05          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.001         | 0.29          | 0.94          | 0.95          |
+------------+---------------+---------------+---------------+---------------+


In [ ]:
model_structure .add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_JohnCall, cpd_MerryCalls)

In [ ]:
model_structure.check_model()

True

In [ ]:
inference= VariableElimination(model_structure)
query_result = inference.query(variables=["Burglary"], evidence={"JohnCalls": 1, "MerryCalls": 1})
print(query_result)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


In [ ]:
#query1
evidence = {'Burglary': 0, 'Earthquake': 0}
result1 = inference.query(variables=["JohnCalls", "MerryCalls", 'Alarm'], evidence=evidence)
print(result1)

+--------------+---------------+----------+-----------------------------------+
| JohnCalls    | MerryCalls    | Alarm    |   phi(JohnCalls,MerryCalls,Alarm) |
+==============+===============+==========+===================================+
| JohnCalls(0) | MerryCalls(0) | Alarm(0) |                            0.9396 |
+--------------+---------------+----------+-----------------------------------+
| JohnCalls(0) | MerryCalls(0) | Alarm(1) |                            0.0000 |
+--------------+---------------+----------+-----------------------------------+
| JohnCalls(0) | MerryCalls(1) | Alarm(0) |                            0.0095 |
+--------------+---------------+----------+-----------------------------------+
| JohnCalls(0) | MerryCalls(1) | Alarm(1) |                            0.0001 |
+--------------+---------------+----------+-----------------------------------+
| JohnCalls(1) | MerryCalls(0) | Alarm(0) |                            0.0495 |
+--------------+---------------+--------

In [ ]:
#query2
query_result_johncalls = inference.query(variables=["JohnCalls"])
print(query_result_johncalls)

+--------------+------------------+
| JohnCalls    |   phi(JohnCalls) |
+==============+==================+
| JohnCalls(0) |           0.9479 |
+--------------+------------------+
| JohnCalls(1) |           0.0521 |
+--------------+------------------+


In [ ]:
#query3
evidence={ 'JohnCalls' :1, 'MerryCalls' : 1 }
result3 = inference.query(variables=["Burglary"], evidence=evidence)
print(result3)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


In [ ]:
# Print the probability of Burglary being True (1)
print(f"Probability of Burglary (True) given JohnCalls=True and MerryCalls=True: {result3.values[1]}")

# Print the probability of Burglary being False (0)
print(f"Probability of Burglary (False) given JohnCalls=True and MerryCalls=True: {result3.values[0]}")

Probability of Burglary (True) given JohnCalls=True and MerryCalls=True: 0.284171835364393
Probability of Burglary (False) given JohnCalls=True and MerryCalls=True: 0.7158281646356071


In [ ]:
#query
query_result_burglary = inference.query(variables=["Burglary"])
print(query_result_burglary)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.9990 |
+-------------+-----------------+
| Burglary(1) |          0.0010 |
+-------------+-----------------+


In [ ]:
#query
query_result_earthquake = inference.query(variables=["Earthquake"])
print(query_result_earthquake)

+---------------+-------------------+
| Earthquake    |   phi(Earthquake) |
+===============+===================+
| Earthquake(0) |            0.9980 |
+---------------+-------------------+
| Earthquake(1) |            0.0020 |
+---------------+-------------------+


In [ ]:
#query 4
query_result_alarm = inference.query(variables=["Alarm"], evidence={"MerryCalls": 1, "JohnCalls": 1})
print(query_result_alarm)

+----------+--------------+
| Alarm    |   phi(Alarm) |
+==========+==============+
| Alarm(0) |       0.2393 |
+----------+--------------+
| Alarm(1) |       0.7607 |
+----------+--------------+


In [ ]:
#query 5
query_result_earthquake_given_calls = inference.query(variables=["Earthquake"], evidence={"JohnCalls": 1, "MerryCalls": 1})
print(query_result_earthquake_given_calls)

+---------------+-------------------+
| Earthquake    |   phi(Earthquake) |
+===============+===================+
| Earthquake(0) |            0.8239 |
+---------------+-------------------+
| Earthquake(1) |            0.1761 |
+---------------+-------------------+



$P(\text{Earthquake=True} \mid \text{JohnCalls=False}, \text{MerryCalls=True})$

In [ ]:
query_result_earthquake_given_john_false_merry_true = inference.query(variables=["Earthquake"], evidence={"JohnCalls": 0, "MerryCalls": 1})
print(query_result_earthquake_given_john_false_merry_true)

+---------------+-------------------+
| Earthquake    |   phi(Earthquake) |
+===============+===================+
| Earthquake(0) |            0.9944 |
+---------------+-------------------+
| Earthquake(1) |            0.0056 |
+---------------+-------------------+


In [ ]:
# Print the probability of Earthquake being True (1)
print(f"Probability of Earthquake (True) given JohnCalls=False and MerryCalls=True: {query_result_earthquake_given_john_false_merry_true.values[1]}")



Probability of Earthquake (True) given JohnCalls=False and MerryCalls=True: 0.005612151520557884


$P(\text{Alarm=False} \mid \text{JohnCalls=True}, \text{MerryCalls=False})$

In [ ]:
query_result_alarm_merryf = inference.query(variables=["Alarm"], evidence={"MerryCalls": 0, "JohnCalls": 1})
print(query_result_alarm_merryf)

+----------+--------------+
| Alarm    |   phi(Alarm) |
+==========+==============+
| Alarm(0) |       0.9864 |
+----------+--------------+
| Alarm(1) |       0.0136 |
+----------+--------------+


In [ ]:


# Print the probability of Alarm being False (0)
print(f"Probability of Alarm (False) given MerryCalls=False and JohnCalls=True: {query_result_alarm_merryf.values[0]}")

Probability of Alarm (False) given MerryCalls=False and JohnCalls=True: 0.9864261106686925
